# Model Manager (모델을 통한 조회) (발표)
Model Manager는 ModelsCls.objects로 제공

Example)    
`SELECT * FROM app_model;`    
`ModelCls.objects.all()`

`SELECT * FROM app_model ORDER BY id DESC LIMIT 10;`    
`ModelCls.objects.all().order_by('-id')[:10]`

`INSERT INTO app_model (title) VALUES ("New Title");`    
`ModelCls.objects.create(title="New Title")`

-(마이너스)를 붙이면 역순으로 설정된다.

In [ ]:
from instagram.models import Post

qs = Post.objects.all() # data를 쿼리할 준비
print(type(qs), qs.query)

# QuerySet
Chaining 지원 -> 조건을 계속해서 덧붙일 수 있다.    
icontains 라는게 포함하느냐는 거다 ...

In [ ]:
query = '검색어'

# 계속해서 덧붙이는 방법
qs = Post.objects.all().filter(message_icontains=query)\
            .order_by('-id') # data를 쿼리할 준비
print(qs) # DB에 접근

# 다양한 조회요청 방법 SELECT ! 
1. 조건 추가시    
queryset.**filter**(year=2005, age=10).**exclude**(color=blue)    
: year이 2005, age가 10 이고, color가 blue인것을 제외한 데이터

2. 특정 모델객체 1개 획득하려고 할때는,    
queryset[index]    
(IndexError 발생 가능) 
```
>>> Entry.objects.all()[:10:2]
>>> Entry.objects.order_by("headline")[0]
```

queryset.get()
get안에 pk=1 과 같은 조건을 넣을 수 있음    
(DoesNoetExist 데이터 없을 때, MultipleOjectsReturned 두 개 이상 요청시  에러 발생 가능)
```
>>> from blog.models import Blog, Entry
>>> entry = Entry.objects.get(pk=1)
>>> cheese_blog = Blog.objects.get(name="Cheddar Talk")
>>> entry.blog = cheese_blog
>>> entry.save()
```

queryset.first()    
(가장 첫번째 것 반환)

queryset.last()    
(가장 마지막 것 반환)

3. Q 사용하기    
연산(& |)이 가능하다.

In [ ]:
from django.db.models import Q

qs = Post.objects.all()
qs = qs.filter(Q(id_gte=2) & Q(message_icontains=query))

cond = Q(id_gte=2) | Q(message_icontains=query)
qs = qs.filter(cond)

3-1. 필드 타입별 다양한 조건 매칭 문법    

숫자, 날짜, 시간과 같은 크기 비교가 가능한 경우,    
- a_lt = 조건값 (a < 조건값) less than
- a_lte = 조건값 (a <= 조건값) less than equal
- a_gt = 조건값 (a > 조건값) greater than
- a_gte = 조건값 (a >= 조건값) greater than equal

문자열 필드의 경우 ,
- a_startswith = 조건값 (a LIKE "조건값%")
- a_lstartswith = 조건값 (a LIKE "조건값%")
- a_endswith = 조건값 (a LIKE "%조건값")
- a_lendswith = 조건값 (a LIKE "%조건값")
- a_contains = 조건값 (a LIKE "%조건값%")
- a_icontains = 조건값 (a LIKE "%조건값%")

# Django로 작업

1. views - item이라는 모델 목록을 가지고 온다.    
request 받고 response를 보내준다.
2. urls - 경로와 views 함수를 연결해준다. 함수를 실행하는게 아니기 때문에 () 붙이면 안됨!
3. 프로젝트의 urls - 각 app의 urls 를 등록해줘야함 

-project (settings, urls에 app 등록)    
-app(models 사용할 모델 설정, views request와 response 설정, urls에서 경로와 해당 경로의 작동 함수 설정)    
-app    
-app    

- `Item.objects.all()` : 모든 리스트를 조회
- request GET, POST, FILES 인자 
`request.GET.get('q', '')` q라는 인자를 가져오고, 없다면 ''를 반환    

- `Item.objects.all().filter(name__incontains=request.GET.get('q', ''))`   
모든 리스트를 filter 하는데, request.GET.get('q', '')를 포함하고 있는 것 

필터링한 결과물 item_list 을 반환하는 예시 :
- `return render(request, '...html', { 'item_list':필터링한결과물, })`    
item_list 로 반환하면 html에서는, {% item_list %} 장고문법으로 표현 가능    

반환하는 이름을 같게 명시하면 된다.

# QuerySet을 정렬해보자 (발표)
SELECT - ORDER BY

가급적 단일 필드를 정렬하는 것이 성능 이익이 있음

**정렬 조건 지정하는 방법**    
1. Model Class - Meta 속성 - ordering 설정
2. 모든 queryset에 order_by(...) 지정 

2를 지정하면 1이 무시됨 

1의 방법, models.py에서 class안에 class 작성,
`class Meta: ordering = ['id']` 라고 작성하면, id 순으로 정렬된다.

[django-extensions](https://django-extensions.readthedocs.io/en/latest/) 를 사용해서        
정렬된 것은, `python manage.py shell_plus --print-sql` 커맨드 실행 시, 
쿼리문이 출력되며 ORDER BY 에 id 가 기입된 것을 볼 수 있다.

str/list/tuple 슬라이싱과 유사, 음수 인덱스 지원안함,    
[start:stop:step]
step까지 기입해서 조회하면 -> 결과가 list type 됨.     
step은 sql이 아닌 django에서 처리해준다.

**역순 슬라이싱**    
Post.objects.all().order_by('id')[:2] 이런식으로 해결!?

# django-debug-toolbar (발표)
request / response 다양한 디버깅 정보 제공
Ajax 요청에 대한 지원은 안함 

다양한 Panel support    
ex) SQL Panel 으로, 각 request 시 발생한 SQL 내역 확인 가능

html에 body 태그가 있어야 동작됨 

실제 쿼리 확인용 - print(Post.objects.all().query)

settings.DEBUG = True 일 때, 쿼리 실행내역이 메모리에 누적되는데,    
그래서 계속 켜두게 되면 메모리 부족 문제가 걸려 서비스에 문제가 생길 수 있다.
베타 테스트라도 False로 해두고 진행해야.

# django-querycount
SQL 실행내역을 개발서버 콘솔 표준출력    
Ajax 내역도 출력 가능

# ORM이란,
SQL 생성을 도와주는 *라이브러리*, DB의 모든 작업 처리는 아님
더 성능좋은 앱을 위해서 DB 엔진 + SQL 에 대한 이해 필요 

### RDBMS 관계
**1:N**
- models.ForeignKey
- 1명 User : 다수 Post(PK, FK)
- 1명 User : 다수 Comment(FK)
- 1개 Post : 다수 Comment(FK)

**1:1**
- model.OneToOneField
- 1 User : 1 Profile(여기에 user와의 관계 정의)
- django에서 user는 auth 앱에서 지원해주는 것
- 커스텀 user도 만들 수 있다. 
- user 와 연결시 user보다는 다른쪽에 관계 정의, django가 만들어 둔 user이므로

**M:N** 
- models.ManyToManyField
- 1 Post : 다수 Tag
- 다수 Post : 1 Tag

### ForeignKey
1: N 일때, N측에 명시한다.
인자 2개 (to, on_delete)

to : 대상모델 (1인 것)

on_delete : record 삭제시 어떻게 할 것인지?

CASCADE : FK로 참조하는 다른 모델의 record도 삭제    
PROTECT : ProtectedError 발생하여 삭제 방지    
SET_NULL : 삭제 시 null로 데이터 대체 , null=True 해둬야함    
SET_DEFAULT : 삭제 시 default값으로 대체    
SET : 대체 값, 함수 지정 가능    
DO_NOTHING 


### User
django의 User 모델을 가져오는 방법

1. 'auth.User'    
`author = models.ForeignKey('auth.User', on_delete=models.CASCADE)`

2. from django.contrib.auth.models import User
`author = models.ForeignKey(User, on_delete=models.CASCADE)`

Custom User 모델 가져오는 방법 (가장 안전, 확실한)

1. setting.py에 프로젝트 만들자마자 작성    
`AUTH_USER_MODEL = 'appname.User'`
(근데 이걸 한번 쓰고 지우는건지? 아니면... 디폴트값으로 되어있다는건지...???후..... ㅡㅡ)    
(이게 써져있으면 에러가 나는데 ㅡㅡ)

In [ ]:
from django.conf import settings

class Post(models.Model):
    author = models.ForeignKey(settings.AUTH_USER_MODEL, on_delete=models.CASCADE)

### FK의 reverse_name
1 : N 의 관계에서 1 측에서 사용하는 것이다.
모델명소문자_set 으로 이름이 생기고, 이름을 변경할 수도 있다.

앱 이름이 아닌 모델명만 고려하기 때문에 reverse_name 이 충돌할 수 있다.

충돌 예시    
blog app, Post model , author = FK(User)     
shop app, Post model , author = FK(User)

이름 충돌 시 makemigrations 명령어 실패

해결방법:
+ 를 붙여 reverse_name을 사용하지 않도록 설정     
author = FK(User+)     
author = FK(User)

related_name을 설정하여 reverse_name 변경 가능    
FK(User, ... , related_name='blog_post_set')     
FK(User, ... , related_name='shop_post_set')

In [ ]:
import os
os.environ['DJANGO_SETTINGS_MODULE']='askcompany.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE']="true"
import django
django.setup()
from instagram.models import Post, Comment
Comment.objects.all()
Comment.objects.all().first()
comment = Comment.objects.first()
# Post.objects.get(pk=comment.post_id)
comment.post

# comment list 불러오기
# post_id가 3인 것
# Comment.objects.filter(post_id=3) 
# 실제가 아닌 관계에 있는???
# Comment.objects.filter(post__id=3)
# Comment.objects.filter(post=post)
post.comment_set.all()

### ForeignKey.limit_choices_to option.
댓글 작성 기능을 사용할 때, 만약 등록할 포스트 리스트를 제한적으로 주고 싶다면

limit_choices_to={'is_staff':True}

어떤 필터링을 주며 어떤걸 나타나게 할건지 설정할 수 있다.

In [ ]:
staff_member = models.ForeignKey(
    User,
    on_delete=models.CASCADE,
    limit_choices_to={'is_public':True},
)